# Modelo Aditivos Generalizados

- Son utilizados principalmente para combatir al underfitting (cuando nuestro modelo es demasiado
rígido, o tiene mucho sesgo/bias)
- Soluciona problemas cuando nuestro modelo genera una linea recta, y en verdad nuestro datos
tienen una curvatura (ej: exponenciales)

## Soluciones a la nolinealidad

- Modelo Lineal: Presenta alto sesgo, pero es fácil de interpretar
- Polinomial: Se acoplan harto a los datos de entrenamiento, pero tienen una chance de generalización
mucho más baja
- GLM: Suaviza/Genera curvas en el modelo hecho. Deja a las variables del modelo en forma lineal
aún así!

## Elementos básicos de GAM - Solución propuesta por GAM

Aquí se utilizan funciones en cada una de las variables. Por ejemplo:

Modelo con 4 variables (A, B, C y D)

y = intercepto + f(A) + f(B) + f(C) + f(D)

Donde f son las funciones que se aplican a cada una de las variables. Por ejemplo: Función identidad
(la variable se deja de la misma forma), función cuadrada (x elevado a 2), función cúbica, función
logaritmica, etc... . Esto hace que nuestro modelo en si siga siendo lineal, pero se introducen
curvaturas a la variable dependiente!

## Implementación y entrenamiento

El objetivo del GAM es encontrar qué función es más óptima para cada uno de las variables del 
modelo. Esto lo realiza utilizando el backfitting:

### ¿Cómo se optimiza el GAM?

Al final, se itera en cada una de las funciones utilizadas para cada variable. Para cada iteración,
se evalua el modelo (por ejemplo, con Mean Squared Error), y...

En cada uno de las funciones utilizadas, se utiliza un penalizador (Utilizando lambda como en 
Ridge o Lasso) para restringir el overfit. Además, se itera por cada variable del modelo para
encontrar el lambda más óptimo!

# Implementación en Python

In [25]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pygam import LinearGAM, s
import lec2_graphs as afx
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error, median_absolute_error

plt.rcParams['figure.figsize'] = (12, 6)
plt.style.use('seaborn')

In [26]:
df = pd.read_csv('kc_house_data.csv')

In [27]:
df = df.drop(['zipcode', 'id', 'date'], axis=1)

In [28]:
sub = df[['bedrooms', 'bathrooms', 'sqft_living', 'sqft_lot']]
X_train_pre, X_test_pre, y_train, y_test = train_test_split(sub, df['price'],
                                                            test_size=0.3,
                                                            random_state=63)

scaler = StandardScaler().fit(X_train_pre)
X_train = pd.DataFrame(scaler.transform(X_train_pre), columns=X_train_pre.columns)
X_test = pd.DataFrame(scaler.transform(X_test_pre), columns=X_test_pre.columns)

In [29]:
lams = np.logspace(-3, 3, 3)
lams = [lams] * len(X_train.columns)
# En la ultima linea, se copia 4 veces (o n veces, segun las variables que se vayan a ocupar
# en el modelo) los lambdas a probar en la regularización de 1 variable!

In [30]:
# s es la funcion con la que se va a modificar cada variable en este modelo. Dentro de la
# documentacion se menciona que s es la funcion cubica. Por lo tanto, todas las variables son
# tratadas con la funcion cubica (Identificar esta funcion creo que tambien es parte del algoritmo
# de GAM)
gam = LinearGAM(s(0) + s(1) + s(2) + s(3), fit_intercept=True)

In [32]:
gam.gridsearch(X_train, y_train, lam=lams)

InvalidIndexError: (slice(None, None, None), 0)

## Dependencia Parcial

Una vez generado el modelo aditivo generalizado, se puede ver el comportamiento de la variable
y con respecto a cada una de las variables. De este modo, es posible observar tendencias en los
distintos valores de cada variable.